# Different Configuration Using Standard Kmeans

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np
import matplotlib.pyplot as plt


from HOTS.Monitor import GenerateAM, DisplayAM
from HOTS.Event import Event, SimpleAlphabet
from HOTS.STS import STS
from HOTS.Layer import ClusteringLayer, FilterNHBD
from HOTS.Monitor import DisplayImage, DisplaySurface3D, DisplaySurface2D, DisplayConvergence
from HOTS.Tools import SaveObject, LoadObject
from HOTS.Classifier import Classifier
from HOTS.Network import Network
from HOTS.Cluster import KmeansMaro

## Defining Common parameter

In [7]:
tau = 0.02
R = 2
nb_cluster = 4
ImageSize = (32,32)
KN=2

## Exp1 :  Standard Kmeans + Linear Kernel +  NoiseFilter(=2) + One Polarity in Input 